# Tutorial 3: Sparse Spyker
We saw how to use spyker in the previous tutorials. Here, we show how to change previous tutorials to use the sparse interface of the library. As before, we import the needed tools first.

In [1]:
import spyker, torch
from spyker import sparse
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

The dataset is loaded like before.

In [2]:
batch = 64
root = './data'
train = MNIST(root, train=True, download=True, transform=ToTensor())
test = MNIST(root, train=False, download=True, transform=ToTensor())
train = DataLoader(train, batch_size=batch)
test = DataLoader(test, batch_size=batch)

Since the sparse interface runs on CPU, we can skip the device arguement in the `Transform` module and change `rankcode` to `rankcvt` to get sparse spikes. 

In [3]:
class Transform:
    def __init__(self):
        self.filter = spyker.LoG(3, [.5, 1, 2], pad=3)
    
    def __call__(self, input):
        return sparse.code(spyker.threshold(self.filter(input), .01), 15)

We skip the device arguement in the `Network` module too. A few other changes need to be applied to the network so that it will be able to process sparse input.

In [4]:
class Network:
    def __init__(self):
        self.conv1 = spyker.Conv(6, 50, 5, pad=2)
        self.conv2 = spyker.Conv(50, 100, 3, pad=1)
        self.conv1.config = spyker.STDPConfig(.004, -.003)
        self.conv2.config = spyker.STDPConfig(.004, -.003)
    
    def train1(self, input):
        output = sparse.inhibit(sparse.convfwd(self.conv1, input, 16))
        sparse.stdp(self.conv1, input, sparse.convwta(output, 3, 5))
        
    def train2(self, input):
        input = sparse.pool(sparse.convfwd(self.conv1, input, 16), 2)
        output = sparse.inhibit(sparse.convfwd(self.conv2, input, 5))
        sparse.stdp(self.conv2, input, sparse.convwta(output, 1, 8))
    
    def __call__(self, input):
        input = sparse.pool(sparse.convfwd(self.conv1, input, 16), 2)
        input = sparse.pool(sparse.convfwd(self.conv2, input, 5), 3)
        return (15 - spyker.to_torch(sparse.gather(input))).flatten(1)

The rest of the code doesn't need to change.

In [5]:
def Total(network, transform, dataset):
    data_total, target_total = [], []
    for data, target in dataset:
        data_total.append(network(transform(data)).cpu())
        target_total.append(target)
    return torch.cat(data_total), torch.cat(target_total)

In [6]:
def Update(config):
    rate = config.neg / config.pos
    pos = min(config.pos * 2, .1)
    config.pos, config.neg = pos, pos * rate

In [7]:
transform = Transform()
network = Network()

In [8]:
from tqdm.notebook import tqdm

for i, (data, _) in enumerate(tqdm(train, "Training Layer 1")):
    if (i + 1) % 10 == 0: Update(network.conv1.config[0])
    network.train1(transform(data))
spyker.quantize(network.conv1.kernel, 0, .5, 1)

for i, (data, _) in enumerate(tqdm(train, "Training Layer 2")):
    if (i + 1) % 10 == 0: Update(network.conv2.config[0])
    network.train2(transform(data))
spyker.quantize(network.conv2.kernel, 0, .5, 1);

Training Layer 1:   0%|          | 0/938 [00:00<?, ?it/s]

Training Layer 2:   0%|          | 0/938 [00:00<?, ?it/s]

In [9]:
train_data, train_target = Total(network, transform, train)
test_data, test_target = Total(network, transform, test)

In [10]:
pca = PCA(n_components=200).fit(train_data, train_target)
train_data, test_data = pca.transform(train_data), pca.transform(test_data)

In [11]:
target = SVC(C=2.4).fit(train_data, train_target).predict(test_data)
accuracy = (torch.tensor(target) == test_target).sum() / len(test_target)
print(f"Final Accuracy: {accuracy * 100 :.2f}%")

Final Accuracy: 99.23%


We can see that the accuracy is nearly the same as before and it runs fairly fast even though it runs on the CPU.